### sql-cheatsheets
![sql cheat sheet page1](sql-page-1.png)

![sql cheat sheet page2](sql-page-2.png)

### Review Banking DB

[schema](banks/schema.sql)

[seed](banks/seed.sql)

### Query

-- Fraud detection: who are the customer paid through Wells Fargo?

```sql
CREATE VIEW payment_summary AS
SELECT p.*, c.first_name, c.last_name, b.bank_name 
FROM payments p
JOIN customer c ON p.customer_id = c.customer_id
JOIN banks b ON p.bank_routing_number = b.bank_routing_number

SELECT * FROM payment_summary
WHERE bank_name = 'Wells Fargo';
```


### Insights

Join connect all the small tables into a big table. 

Questions, why do we just create one gaint table include everything, like the one that we have joined all the tables?

tips:
use scratch pad, paste the schema on top.
use the line only for one command


[Sakila DB](https://dev.mysql.com/doc/sakila/en/sakila-structure-tables.html)

# explain the data set. the db is the business structure. query for insights
- Sakila db 

### Join vs subquery

```sql

  
-- Identify customers who rented the film 'PANIC CLUB'.
SELECT DISTINCT c.customer_id, c.first_name, c.last_name
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
WHERE f.title = 'PANIC CLUB';

SELECT c.customer_id, c.first_name, c.last_name
FROM customer c
WHERE c.customer_id IN (
    SELECT r.customer_id
    FROM rental r
    WHERE r.inventory_id IN (
        SELECT i.inventory_id
        FROM inventory i
        WHERE i.film_id IN (
            SELECT f.film_id
            FROM film f
            WHERE f.title = 'PANIC CLUB'
        )
    )
);

-- Calculate the total rental revenue from the 'Action' category.
SELECT SUM(p.amount) AS total_revenue
FROM payment p
JOIN rental r ON p.rental_id = r.rental_id
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
WHERE c.name = 'Action';

SELECT SUM(p.amount) AS total_revenue
FROM payment p
WHERE p.rental_id IN (
    SELECT r.rental_id
    FROM rental r
    WHERE r.inventory_id IN (
        SELECT i.inventory_id
        FROM inventory i
        WHERE i.film_id IN (
            SELECT f.film_id
            FROM film f
            JOIN film_category fc ON f.film_id = fc.film_id
            JOIN category c ON fc.category_id = c.category_id
            WHERE c.name = 'Action'
        )
    )
);

-- Find the list of films that have never been rented.
SELECT f.film_id, f.title
FROM film f
LEFT JOIN inventory i ON f.film_id = i.film_id
LEFT JOIN rental r ON i.inventory_id = r.inventory_id
WHERE r.rental_id IS NULL;

SELECT f.film_id, f.title
FROM film f
WHERE NOT EXISTS (
    SELECT 1
    FROM rental r
    JOIN inventory i ON r.inventory_id = i.inventory_id
    WHERE i.film_id = f.film_id
);

```

#
- everything is a table.
- how to make the table bigger (join)
- how to maket the table smaller (group by)
- combine join and where in

# steps
- think about the tables (relations) getting involved
- think about the cols 
- structure the query on the high level
- test on partial querys and build on these querys. 1. combine join, 2. where in, 3. group by

```sql
--1. How many customers in total?
SELECT COUNT(*) AS total_customers
FROM customer;
--2. What is the total amount of payment received?
SELECT SUM(amount) AS total_revenue
FROM payment;
--3. How many films in its inventory?
SELECT COUNT(*) AS total_films
FROM film;
--4. What is the average rental rate for the films?
SELECT AVG(rental_rate) AS average_rental_rate
FROM film;
--5. List the top 10 customers who have made the most number of rentals.
SELECT c.customer_id, c.first_name, c.last_name, COUNT(r.rental_id) as rental_count
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
GROUP BY c.customer_id
ORDER BY rental_count DESC
LIMIT 10;
--6. What are the top 10 most rented films?
SELECT f.film_id, f.title, COUNT(r.rental_id) as rental_count
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.film_id
ORDER BY rental_count DESC
LIMIT 10;
--7. Calculate the total revenue for each store
SELECT s.store_id, SUM(p.amount) as total_revenue
FROM store s
JOIN staff st ON s.store_id = st.store_id
JOIN payment p ON st.staff_id = p.staff_id
GROUP BY s.store_id;
--8. List the top 10 customers who have spent the most on rentals.
SELECT c.customer_id, c.first_name, c.last_name, SUM(p.amount) as total_spent
FROM customer c
JOIN payment p ON c.customer_id = p.customer_id
GROUP BY c.customer_id
ORDER BY total_spent DESC
LIMIT 10;
--9. Find films that have never been rented.
SELECT f.film_id, f.title
FROM film f
WHERE f.film_id NOT IN (
    SELECT DISTINCT i.film_id
    FROM rental r
    JOIN inventory i ON r.inventory_id = i.inventory_id
);
```
